## Set Up

In [1]:
import huggingface_hub

In [2]:
from huggingface_hub import notebook_login

notebook_login() #https://huggingface.co/settings/tokens

In [3]:
from datasets import load_dataset

In [4]:
books = load_dataset("grosenthal/latin_english_parallel")

In [5]:
books['train'], books['test']

(Dataset({
     features: ['id', 'la', 'en', 'file'],
     num_rows: 99343
 }),
 Dataset({
     features: ['id', 'la', 'en', 'file'],
     num_rows: 1014
 }))

In [6]:
## TEMPORARY: to save training time, use just % of the training set first, change `prop_of_training_use` to 1 later when more powerful machine is accessible
prop_of_training_use = 0.2

index_list = list(range(len(books['train'])))
split_index = int(len(books['train']) * prop_of_training_use)
set1_indices = index_list[:split_index]
set2_indices = index_list[split_index:]
print(len(set1_indices), len(set2_indices))

# Split dataset into set1 and set2
set1 = books['train'].select(set1_indices)
set2 = books['train'].select(set2_indices)

books['train'] = set1 

19868 79475


In [7]:
len(books['train']), len(books['test'])

(19868, 1014)

## Preprocess

In [8]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
source_lang = "en"
target_lang = "la"
prefix = "Translate English to Latin: "


def preprocess_function(examples):
    inputs = [prefix + text for text in examples[source_lang]]
    targets = [text for text in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

In [11]:
import evaluate

metric = evaluate.load("sacrebleu")

In [12]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
model_name = "translator-en-la"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_name,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    # fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/home/codespace/.python/current/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()
trainer.push_to_hub()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,4.517000,4.156917,0.125300,18.931000
2,4.333600,4.061716,0.121000,18.932000


/home/codespace/.python/current/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MenaWANG/translator-en-la/commit/82295f411a77a16fd30f3f762cd6dfae1246ee52', commit_message='End of training', commit_description='', oid='82295f411a77a16fd30f3f762cd6dfae1246ee52', pr_url=None, pr_revision=None, pr_num=None)

## Inference

In [16]:
text = "Translate English to Latin: For God so loved the world that he gave his one and only Son, that whoever believes in him shall not perish but have eternal life."

In [17]:
from transformers import pipeline 

translator = pipeline("translation", model=model_name)
translator(text)

/home/codespace/.python/current/lib/python3.10/site-packages/transformers/pipelines/__init__.py:1086: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'ad etiam ad etiam etiam etiam etiam etiam etiam etiam etiam etiam etiam'}]